In [1]:
import pandas as pd
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
import skimage
import glob
import numpy
from sklearn.metrics.pairwise import cosine_similarity
from skimage.measure import compare_ssim
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import mutual_info_score
from PIL import Image
from keras.models import load_model
from keras import backend as K
#K.set_image_dim_ordering('tf')
import joblib
#K.image_data_format()

Using TensorFlow backend.


In [2]:
filepath = 'fer2013.csv'

In [3]:
img_rows, img_cols = 48, 48

data = pd.read_csv(filepath)
data = data.drop(data.index[len(data)-1])

In [4]:
data.shape

(35886, 3)

In [5]:
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [6]:
#Convert the data format from the original format to a 48x48 array.
#Original format is the pixel values stored in a string, where each value is seperated by a space.
def convert_data(data):
    
    X_train = data['pixels'][0].split(' ')
    X_train = np.asarray(X_train).astype('float32')
    X_train = X_train.reshape(1,48,48)
    
    Y_train = data['emotion']
    Y_train = np.asarray(Y_train)
    
    for i in range(1, data.shape[0]):
        
        x = data['pixels'][i].split(' ')
        x = np.asarray(x).astype('float32')
        x = x.reshape(1,48,48)
        X_train = np.concatenate((X_train,x), axis=0)

    
    return [X_train, Y_train]

In [7]:
def get_mean(X):
    
    mean = np.mean(X)
    
    return mean

In [8]:
def get_std(X):
    
    std = np.std(X)
    return std

In [9]:
#Save the converted dataset
#X, Y = convert_data(data)

#joblib.dump(X, 'X.pkl')
#joblib.dump(Y, 'Y.pkl')

#Load the saved dataset
X = joblib.load('X.pkl')
Y = joblib.load('Y.pkl')

In [10]:
X.shape

(35886, 48, 48)

In [11]:
def convert_img(img):
    img = imgToarr(img)

In [12]:
#Function converts the image to a numpy array
def imgToarr(img):
    
    return numpy.array(img)

In [13]:
#Resizes the image to the specified dimensions
def resize(img,x,y):
    
    return img.resize((x,y),Image.ANTIALIAS)

In [14]:
#Converting a N-Dimensional array to 1-D array
def reshape_1D(arr):
    
    return arr.ravel()

In [15]:
def list_to_numpyarray(convert_list):
    
    return np.asarray(convert_list)

In [16]:
X_train = X[0:28709]
Y_train = Y[0:28709]
X_test = X[28709:]
Y_test = Y[28709:]

In [17]:
Y_train = Y_train.reshape(Y_train.shape[0],1)
Y_test = Y_test.reshape(Y_test.shape[0],1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train -= np.mean(X_train)
X_train /= np.std(X_train)
X_test -= np.mean(X_test)
X_test /= np.std(X_test)

In [18]:
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

In [19]:
nb_classes = 7
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

In [20]:
#Initializing the values for the convolution neural network
nb_epoch = 100
batch_size = 64

In [ ]:
model = Sequential()

model.add(Convolution2D(64, (3, 3), padding='same',
                        input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-5),
              metrics=['accuracy'])

filepath="emotion1.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch, callbacks=callbacks_list, \
          validation_data=(X_test, Y_test),shuffle=True)

Train on 28709 samples, validate on 7177 samples
Epoch 1/100
28709/28709 [==============================] - 782s 27ms/step - loss: 1.8123 - acc: 0.2492 - val_loss: 1.7793 - val_acc: 0.2560
Epoch 2/100
28709/28709 [==============================] - 1018s 35ms/step - loss: 1.7721 - acc: 0.2626 - val_loss: 1.7527 - val_acc: 0.2760
Epoch 3/100
28709/28709 [==============================] - 824s 29ms/step - loss: 1.7219 - acc: 0.3059 - val_loss: 1.6654 - val_acc: 0.3451
Epoch 4/100
28709/28709 [==============================] - 790s 28ms/step - loss: 1.6257 - acc: 0.3617 - val_loss: 1.5753 - val_acc: 0.3911
Epoch 5/100
28709/28709 [==============================] - 818s 28ms/step - loss: 1.5535 - acc: 0.3976 - val_loss: 1.5217 - val_acc: 0.4243
Epoch 6/100
28709/28709 [==============================] - 892s 31ms/step - loss: 1.5139 - acc: 0.4194 - val_loss: 1.4975 - val_acc: 0.4240
Epoch 7/100
25920/28709 [==========================>...] - ETA: 1:27 - loss: 1.4854 - acc: 0.4321